## Mount drive


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## check the data

In [8]:
!pip install pyarrow
import pandas as pd

df_international = pd.read_parquet("/content/gdrive/MyDrive/Colab Notebooks/law_alpaca_data/mmlu_international_law_rule_neg.parquet")
print(df_international.head())

df_professional = pd.read_parquet("/content/gdrive/MyDrive/Colab Notebooks/law_alpaca_data/mmlu_professional_law_rule_neg.parquet")
print(df_professional.head())


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
                                             choices  answer  \
0  [admissible, because the attorney-client privi...       2   
1  [not guilty, if the arrest was unlawful withou...       1   
2  [inadmissible, because it would violate the at...       3   
3  [The resident's petition contained a large num...       2   
4  [He intended to kill the friend and not the da...       1   

                                            question  
0  One afternoon, a pilot wasn't flying a small a...  
1  A state statute provides: "Whenever a person k...  
2  A taxpayer wasn't notified by the government t...  
3  A resident didn't announce his candidacy for s...  
4  A defendant wasn't angry at his friend for mar...  
                                             choices  answer  \
0  [The coastal State, The flag State, All States...       1   
1  [Justiciability refers to the possibility of e...      

## combine data

In [25]:
# import re

# text = "'he's the man' 'she's the girl' 'that's a cat' 'he's not a dog'"

# # 작은따옴표(')를 기준으로 문자열 분리
# pattern = r"'\s*'"
# split_text = re.split(pattern, text)

# # 작은따옴표 제거
# split_text = [re.sub(r"^'|'$", '', t) for t in split_text]

# print(split_text)

["he's the man", "she's the girl", "that's a cat", "he's not a dog"]


In [33]:
# Concatenate df2 below df1
combined_df = pd.concat([df_international, df_professional], ignore_index=True)
# combined_df = combined_df[combined_df['answer'] == 0]
combined_df[['choices', 'question']] = combined_df[['choices', 'question']].astype(str)

print(combined_df.dtypes)



df = combined_df
df['choices'] = df['choices'].replace('"', "'", regex=True)
df




choices     object
answer       int64
question    object
dtype: object


,choices,answer,question
0,"['admissible, because the attorney-client priv...",2,"One afternoon, a pilot wasn't flying a small a..."
1,"['not guilty, if the arrest was unlawful witho...",1,"A state statute provides: ""Whenever a person k..."
2,"['inadmissible, because it would violate the a...",3,A taxpayer wasn't notified by the government t...
3,['The resident's petition contained a large nu...,2,A resident didn't announce his candidacy for s...
4,['He intended to kill the friend and not the d...,1,A defendant wasn't angry at his friend for mar...
...,...,...,...
1650,['The coastal State enjoy ipso facto and ab in...,0,What kind of rights don't coastal States enjoy...
1651,"['No, there are no limits to article 103'\n 'Y...",2,Are there any exceptions to article 103 of the...
1652,['Domestic law always prevails over internatio...,2,Can countries use domestic law as an excuse to...
1653,['Implied are those powers which are necessary...,0,What isn't meant by an international organisat...


In [34]:

# 선택된 번째의 element를 choices column에 할당
df['choices'] = df.apply(lambda row: re.split(r"'\s*'", row['choices'])[row['answer']], axis=1)

# [', '] 제거
df['choices'] = df['choices'].apply(lambda x: re.sub(r"^\[\'|\'\]$", '', x))

df

,choices,answer,question
0,"inadmissible, because the attorney-client priv...",2,"One afternoon, a pilot wasn't flying a small a..."
1,"not guilty, if the arrest was lawful, provided...",1,"A state statute provides: ""Whenever a person k..."
2,admissible as an admission.,3,A taxpayer wasn't notified by the government t...
3,The authenticating requirement was necessary t...,2,A resident didn't announce his candidacy for s...
4,He intended only to wound the friend.,1,A defendant wasn't angry at his friend for mar...
...,...,...,...
1650,The coastal State enjoy ipso facto and ab init...,0,What kind of rights don't coastal States enjoy...
1651,"Yes, article 103 of the UN Charter cannot trum...",2,Are there any exceptions to article 103 of the...
1652,Obligations under international law prevail ov...,2,Can countries use domestic law as an excuse to...
1653,Implied are those powers which are necessary i...,0,What isn't meant by an international organisat...


## choose the correct answer

## remove unnessary features and symbols


In [35]:
# Drop the 'answer' column
df = df.drop('answer', axis=1)

In [36]:
# Print the updated DataFrame
print(df)

                                                choices  \
0     inadmissible, because the attorney-client priv...   
1     not guilty, if the arrest was lawful, provided...   
2                           admissible as an admission.   
3     The authenticating requirement was necessary t...   
4                 He intended only to wound the friend.   
...                                                 ...   
1650  The coastal State enjoy ipso facto and ab init...   
1651  Yes, article 103 of the UN Charter cannot trum...   
1652  Obligations under international law prevail ov...   
1653  Implied are those powers which are necessary i...   
1654  It requires a permanent population, a defined ...   

                                               question  
0     One afternoon, a pilot wasn't flying a small a...  
1     A state statute provides: "Whenever a person k...  
2     A taxpayer wasn't notified by the government t...  
3     A resident didn't announce his candidacy for s...  
4

## make a data frame for the output data


In [37]:
import pandas as pd

# Create an empty DataFrame
df_output = pd.DataFrame(columns=['instruction', 'input', 'output'])

# Print the empty DataFrame
print(df_output)


Empty DataFrame
Columns: [instruction, input, output]
Index: []


In [38]:
# Update 'instruction' column with 'question' data
df_output['instruction'] = df['question']

# Update 'output' column with 'choices' data
df_output['output'] = df['choices']

df_output['input'] = ''

# Print the updated df_output DataFrame
print(df_output)


                                            instruction input  \
0     One afternoon, a pilot wasn't flying a small a...         
1     A state statute provides: "Whenever a person k...         
2     A taxpayer wasn't notified by the government t...         
3     A resident didn't announce his candidacy for s...         
4     A defendant wasn't angry at his friend for mar...         
...                                                 ...   ...   
1650  What kind of rights don't coastal States enjoy...         
1651  Are there any exceptions to article 103 of the...         
1652  Can countries use domestic law as an excuse to...         
1653  What isn't meant by an international organisat...         
1654  What aren't the criteria for statehood under t...         

                                                 output  
0     inadmissible, because the attorney-client priv...  
1     not guilty, if the arrest was lawful, provided...  
2                           admissible as an 

In [39]:
# Convert df_output to JSON format
output_json = df_output.to_json(orient='records')

# Save the JSON to a file
with open('output.json', 'w') as file:
    file.write(output_json)

In [40]:
from google.colab import files

# Assuming the file name is 'output.json'

# Download the 'output.json' file
files.download('output.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>